In [1]:
from math import sqrt
from numpy import load, asarray, zeros, ones, savez_compressed
from numpy.random import randn, randint
from skimage.transform import resize
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape, Conv2D
from tensorflow.keras.layers import UpSampling2D, AveragePooling2D, LeakyReLU, Layer, Add
from keras.constraints import max_norm
from keras.initializers import RandomNormal
import mtcnn
from mtcnn.mtcnn import MTCNN
from matplotlib import pyplot
import cv2
import os
from os import listdir
from PIL import Image
import cv2
import tensorflow as tf

backend = tf.keras.backend
K = tf.keras.backend

In [16]:

from PIL import Image
import os
from mtcnn.mtcnn import MTCNN
from numpy import load, asarray, zeros, ones, savez_compressed

In [17]:
# Loading the image file
def load_image(filename):
    image = Image.open(filename)
    image = image.convert('RGB')
    pixels = asarray(image)
    return pixels
    
# extract the face from a loaded image and resize
def extract_face(model, pixels, required_size=(128, 128)):
    # detect face in the image
    faces = model.detect_faces(pixels)
    if len(faces) == 0:
        return None
    
    # extract details of the face
    x1, y1, width, height = faces[0]['box']
    x1, y1 = abs(x1), abs(y1)
    
    x2, y2 = x1 + width, y1 + height
    face_pixels = pixels[y1:y2, x1:x2]
    image = Image.fromarray(face_pixels)
    image = image.resize(required_size)
    face_array = asarray(image)
    
    return face_array
    
# load images and extract faces for all images in a directory
def load_faces(directory, n_faces):
    # prepare model
    model = MTCNN()
    faces = list()
    
    for filename in os.listdir(directory):
        # Computing the retrieval and extraction of faces
        pixels = load_image(directory + filename)
        face = extract_face(model, pixels)
        if face is None:
            continue
        faces.append(face)
        print(len(faces), face.shape)
        if len(faces) >= n_faces:
            break
            
    return asarray(faces)

In [18]:
# load and extract all faces
directory = 'img_align_celeba/'
all_faces = load_faces(directory, 100)
print('Loaded: ', all_faces.shape)

# save in compressed format
savez_compressed('img_align_celeba_128.npz', all_faces)

1 (128, 128, 3)
2 (128, 128, 3)
3 (128, 128, 3)
4 (128, 128, 3)
5 (128, 128, 3)
6 (128, 128, 3)
7 (128, 128, 3)
8 (128, 128, 3)
9 (128, 128, 3)
10 (128, 128, 3)
11 (128, 128, 3)
12 (128, 128, 3)
13 (128, 128, 3)
14 (128, 128, 3)
15 (128, 128, 3)
16 (128, 128, 3)
17 (128, 128, 3)
18 (128, 128, 3)
19 (128, 128, 3)
20 (128, 128, 3)
21 (128, 128, 3)
22 (128, 128, 3)
23 (128, 128, 3)
24 (128, 128, 3)
25 (128, 128, 3)
26 (128, 128, 3)
27 (128, 128, 3)
28 (128, 128, 3)
29 (128, 128, 3)
30 (128, 128, 3)
31 (128, 128, 3)
32 (128, 128, 3)
33 (128, 128, 3)
34 (128, 128, 3)
35 (128, 128, 3)
36 (128, 128, 3)
37 (128, 128, 3)
38 (128, 128, 3)
39 (128, 128, 3)
40 (128, 128, 3)
41 (128, 128, 3)
42 (128, 128, 3)
43 (128, 128, 3)
44 (128, 128, 3)
45 (128, 128, 3)
46 (128, 128, 3)
47 (128, 128, 3)
48 (128, 128, 3)
49 (128, 128, 3)
50 (128, 128, 3)
51 (128, 128, 3)
52 (128, 128, 3)
53 (128, 128, 3)
54 (128, 128, 3)
55 (128, 128, 3)
56 (128, 128, 3)
57 (128, 128, 3)
58 (128, 128, 3)
59 (128, 128, 3)
60 (12

In [4]:
# load the prepared dataset
from numpy import load
data = load('img_align_celeba_128.npz')
faces = data['arr_0']
print('Loaded: ', faces.shape)

Loaded:  (1000, 128, 128, 3)


In [5]:
# pixel-wise feature vector normalization layer
class PixelNormalization(Layer):
    # initialize the layer
    def __init__(self, **kwargs):
        super(PixelNormalization, self).__init__(**kwargs)
 
    # perform the operation
    def call(self, inputs):
        # computing pixel values
        values = inputs**2.0
        mean_values = backend.mean(values, axis=-1, keepdims=True)
        mean_values += 1.0e-8
        l2 = backend.sqrt(mean_values)
        normalized = inputs / l2
        return normalized
 
    # define the output shape of the layer
    def compute_output_shape(self, input_shape):
        return input_shape

In [6]:
# mini-batch standard deviation layer
class MinibatchStdev(Layer):
    # initialize the layer
    def __init__(self, **kwargs):
        super(MinibatchStdev, self).__init__(**kwargs)
 
    # perform the operation
    def call(self, inputs):
        mean = backend.mean(inputs, axis=0, keepdims=True)
        squ_diffs = backend.square(inputs - mean)
        mean_sq_diff = backend.mean(squ_diffs, axis=0, keepdims=True)
        mean_sq_diff += 1e-8
        stdev = backend.sqrt(mean_sq_diff)
        
        mean_pix = backend.mean(stdev, keepdims=True)
        shape = backend.shape(inputs)
        output = backend.tile(mean_pix, (shape[0], shape[1], shape[2], 1))
        
        combined = backend.concatenate([inputs, output], axis=-1)
        return combined
 
    # define the output shape of the layer
    def compute_output_shape(self, input_shape):
        input_shape = list(input_shape)
        input_shape[-1] += 1
        return tuple(input_shape)

In [7]:
# weighted sum output
class WeightedSum(Add):
    # init with default value
    def __init__(self, alpha=0.0, **kwargs):
        super(WeightedSum, self).__init__(**kwargs)
        self.alpha = tf.Variable(alpha, dtype=tf.float32, trainable=False, name='ws_alpha')
 
    # output a weighted sum of inputs
    def _merge_function(self, inputs):
        # only supports a weighted sum of two inputs
        assert (len(inputs) == 2)
        # ((1-a) * input1) + (a * input2)
        output = ((1.0 - self.alpha) * inputs[0]) + (self.alpha * inputs[1])
        return output

# calculate wasserstein loss
def wasserstein_loss(y_true, y_pred):
    return K.mean(y_true * y_pred)

In [8]:
# load dataset
def load_real_samples(filename):
    data = load(filename)
    X = data['arr_0']
    X = X.astype('float32')
    X = (X - 127.5) / 127.5
    return X
 
# select real samples
def generate_real_samples(dataset, n_samples):
    ix = randint(0, dataset.shape[0], n_samples)
    X = dataset[ix]
    y = ones((n_samples, 1))
    return X, y
 
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    x_input = randn(latent_dim * n_samples)
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input
 
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
    x_input = generate_latent_points(latent_dim, n_samples)
    X = generator.predict(x_input)
    y = -ones((n_samples, 1))
    return X, y
 
# update the alpha value on each instance of WeightedSum
def update_fadein(models, step, n_steps):
    alpha = step / float(n_steps - 1)
    for model in models:
        for layer in model.layers:
            if isinstance(layer, WeightedSum):
                backend.set_value(layer.alpha, alpha)
                
# scale images to preferred size
def scale_dataset(images, new_shape):
    images_list = list()
    for image in images:
        new_image = resize(image, new_shape, 0)
        images_list.append(new_image)
    return asarray(images_list)

In [9]:
# adding a generator block
def add_generator_block(old_model):
    init = RandomNormal(stddev=0.02)
    const = max_norm(1.0)
    block_end = old_model.layers[-2].output
    
    # upsample, and define new block
    upsampling = UpSampling2D()(block_end)
    g = Conv2D(128, (3,3), padding='same', kernel_initializer=init, kernel_constraint=const)(upsampling)
    g = PixelNormalization()(g)
    g = LeakyReLU(alpha=0.2)(g)
    g = Conv2D(128, (3,3), padding='same', kernel_initializer=init, kernel_constraint=const)(g)
    g = PixelNormalization()(g)
    g = LeakyReLU(alpha=0.2)(g)
    
    out_image = Conv2D(3, (1,1), padding='same', kernel_initializer=init, kernel_constraint=const)(g)
    model1 = Model(old_model.input, out_image)
    out_old = old_model.layers[-1]
    out_image2 = out_old(upsampling)
    
    merged = WeightedSum()([out_image2, out_image])
    model2 = Model(old_model.input, merged)
    return [model1, model2]

In [10]:
# define generator models
def define_generator(latent_dim, n_blocks, in_dim=4):
    init = RandomNormal(stddev=0.02)
    const = max_norm(1.0)
    model_list = list()
    in_latent = Input(shape=(latent_dim,))
    g  = Dense(128 * in_dim * in_dim, kernel_initializer=init, kernel_constraint=const)(in_latent)
    g = Reshape((in_dim, in_dim, 128))(g)
    
    # conv 4x4, input block
    g = Conv2D(128, (3,3), padding='same', kernel_initializer=init, kernel_constraint=const)(g)
    g = PixelNormalization()(g)
    g = LeakyReLU(alpha=0.2)(g)
    
    # conv 3x3
    g = Conv2D(128, (3,3), padding='same', kernel_initializer=init, kernel_constraint=const)(g)
    g = PixelNormalization()(g)
    g = LeakyReLU(alpha=0.2)(g)
    
    # conv 1x1, output block
    out_image = Conv2D(3, (1,1), padding='same', kernel_initializer=init, kernel_constraint=const)(g)
    model = Model(in_latent, out_image)
    model_list.append([model, model])
    
    for i in range(1, n_blocks):
        old_model = model_list[i - 1][0]
        models = add_generator_block(old_model)
        model_list.append(models)
        
    return model_list

In [11]:
# adding a discriminator block
def add_discriminator_block(old_model, n_input_layers=3):
    init = RandomNormal(stddev=0.02)
    const = max_norm(1.0)
    in_shape = list(old_model.input.shape)
    
    # define new input shape as double the size
    input_shape = (in_shape[-2]*2, in_shape[-2]*2, in_shape[-1])
    in_image = Input(shape=input_shape)
    
    # define new input processing layer
    d = Conv2D(128, (1,1), padding='same', kernel_initializer=init, kernel_constraint=const)(in_image)
    d = LeakyReLU(alpha=0.2)(d)
    
    # define new block
    d = Conv2D(128, (3,3), padding='same', kernel_initializer=init, kernel_constraint=const)(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(128, (3,3), padding='same', kernel_initializer=init, kernel_constraint=const)(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = AveragePooling2D(pool_size=(2,2))(d)

    block_new = d
    
    # skip the input, 1x1 and activation for the old model
    for i in range(n_input_layers, len(old_model.layers)):
        d = old_model.layers[i](d)
    model1 = Model(in_image, d)
    
    model1.compile(loss=wasserstein_loss, optimizer=Adam(learning_rate=0.001, beta_1=0, beta_2=0.99, epsilon=10e-8))
    
    downsample = AveragePooling2D(pool_size=(2,2))(in_image)
    
    block_old = old_model.layers[1](downsample)
    block_old = old_model.layers[2](block_old)
    d = WeightedSum()([block_old, block_new])
    
    for i in range(n_input_layers, len(old_model.layers)):
        d = old_model.layers[i](d)
        
    model2 = Model(in_image, d)
    
    model2.compile(loss=wasserstein_loss, optimizer=Adam(learning_rate=0.001, beta_1=0, beta_2=0.99, epsilon=10e-8))
    return [model1, model2]

In [12]:
# define the discriminator models for each image resolution
def define_discriminator(n_blocks, input_shape=(4,4,3)):
    init = RandomNormal(stddev=0.02)
    const = max_norm(1.0)
    model_list = list()
    in_image = Input(shape=input_shape)
    
    d = Conv2D(128, (1,1), padding='same', kernel_initializer=init, kernel_constraint=const)(in_image)
    d = LeakyReLU(alpha=0.2)(d)
    d = MinibatchStdev()(d)
    
    d = Conv2D(128, (3,3), padding='same', kernel_initializer=init, kernel_constraint=const)(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(128, (4,4), padding='same', kernel_initializer=init, kernel_constraint=const)(d)
    d = LeakyReLU(alpha=0.2)(d)
    
    d = Flatten()(d)
    out_class = Dense(1)(d)
    
    model = Model(in_image, out_class)
    model.compile(loss=wasserstein_loss, optimizer=Adam(learning_rate=0.001, beta_1=0, beta_2=0.99, epsilon=10e-8))
    model_list.append([model, model])
    
    for i in range(1, n_blocks):
        old_model = model_list[i - 1][0]
        models = add_discriminator_block(old_model)
        model_list.append(models)
        
    return model_list

In [13]:
# define composite models for training generators via discriminators

def define_composite(discriminators, generators):
    model_list = list()
    # create composite models
    for i in range(len(discriminators)):
        g_models, d_models = generators[i], discriminators[i]
        # straight-through model
        d_models[0].trainable = False
        model1 = Sequential()
        model1.add(g_models[0])
        model1.add(d_models[0])
        model1.compile(loss=wasserstein_loss, optimizer=Adam(learning_rate=0.001, beta_1=0, beta_2=0.99, epsilon=10e-8))
        # fade-in model
        d_models[1].trainable = False
        model2 = Sequential()
        model2.add(g_models[1])
        model2.add(d_models[1])
        model2.compile(loss=wasserstein_loss, optimizer=Adam(learning_rate=0.001, beta_1=0, beta_2=0.99, epsilon=10e-8))
        # store
        model_list.append([model1, model2])
    return model_list

In [14]:
# generate samples and save as a plot and save the model
def summarize_performance(status, g_model, latent_dim, n_samples=25):
    gen_shape = g_model.output_shape
    name = '%03dx%03d-%s' % (gen_shape[1], gen_shape[2], status)
    
    X, _ = generate_fake_samples(g_model, latent_dim, n_samples)
    X = (X - X.min()) / (X.max() - X.min())
    
    square = int(sqrt(n_samples))
    for i in range(n_samples):
        pyplot.subplot(square, square, 1 + i)
        pyplot.axis('off')
        pyplot.imshow(X[i])
        
    # save plot to file
    filename1 = 'plot_%s.png' % (name)
    pyplot.savefig(filename1)
    pyplot.close()
    
    filename2 = 'model_%s.h5' % (name)
    g_model.save(filename2)
    print('>Saved: %s and %s' % (filename1, filename2))

In [ ]:
# train a generator and discriminator
def train_epochs(g_model, d_model, gan_model, dataset, n_epochs, n_batch, fadein=False):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    n_steps = bat_per_epo * n_epochs
    half_batch = int(n_batch / 2)
    
    for i in range(n_steps):
        # update alpha for all WeightedSum layers when fading in new blocks
        if fadein:
            update_fadein([g_model, d_model, gan_model], i, n_steps)
        # prepare real and fake samples
        X_real, y_real = generate_real_samples(dataset, half_batch)
        X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
        
        # update discriminator model
        d_loss1 = d_model.train_on_batch(X_real, y_real)
        d_loss2 = d_model.train_on_batch(X_fake, y_fake)
        
        # update the generator via the discriminator's error
        z_input = generate_latent_points(latent_dim, n_batch)
        y_real2 = ones((n_batch, 1))
        g_loss = gan_model.train_on_batch(z_input, y_real2)
        
        # Extract scalar values
        d_loss1 = d_loss1[0] if isinstance(d_loss1, list) else d_loss1
        d_loss2 = d_loss2[0] if isinstance(d_loss2, list) else d_loss2
        g_loss = g_loss[0] if isinstance(g_loss, list) else g_loss
        
        # summarize loss on this batch
        print('>%d, d1=%.3f, d2=%.3f g=%.3f' % (i+1, d_loss1, d_loss2, g_loss))
        
# train the generator and discriminator
def train(g_models, d_models, gan_models, dataset, latent_dim, e_norm, e_fadein, n_batch):
    g_normal, d_normal, gan_normal = g_models[0][0], d_models[0][0], gan_models[0][0]
    gen_shape = g_normal.output_shape
    scaled_data = scale_dataset(dataset, gen_shape[1:])
    print('Scaled Data', scaled_data.shape)

    # train normal or straight-through models
    train_epochs(g_normal, d_normal, gan_normal, scaled_data, e_norm[0], n_batch[0])
    summarize_performance('tuned', g_normal, latent_dim)
    
    # process each level of growth
    for i in range(1, len(g_models)):
        # retrieve models for this level of growth
        [g_normal, g_fadein] = g_models[i]
        [d_normal, d_fadein] = d_models[i]
        [gan_normal, gan_fadein] = gan_models[i]
        
        # scale dataset to appropriate size
        gen_shape = g_normal.output_shape
        scaled_data = scale_dataset(dataset, gen_shape[1:])
        print('Scaled Data', scaled_data.shape)
        
        # train fade-in models for next level of growth
        train_epochs(g_fadein, d_fadein, gan_fadein, scaled_data, e_fadein[i], n_batch[i], True)
        summarize_performance('faded', g_fadein, latent_dim)
        
        # train normal or straight-through models
        train_epochs(g_normal, d_normal, gan_normal, scaled_data, e_norm[i], n_batch[i])
        summarize_performance('tuned', g_normal, latent_dim)

: 

In [ ]:
# number of growth phases where 6 blocks == [4, 8, 16, 32, 64, 128]
n_blocks = 6
latent_dim = 100

d_models = define_discriminator(n_blocks)
g_models = define_generator(latent_dim, n_blocks)
gan_models = define_composite(d_models, g_models)

dataset = load_real_samples('img_align_celeba_128.npz')
print('Loaded', dataset.shape)

n_batch = [16, 16, 16, 8, 4, 4]
n_epochs = [5, 8, 8, 10, 10, 10]

train(g_models, d_models, gan_models, dataset, latent_dim, n_epochs, n_epochs, n_batch)

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Loaded (1000, 128, 128, 3)
Scaled Data (1000, 4, 4, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step


/opt/anaconda3/lib/python3.12/site-packages/keras/src/backend/tensorflow/trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


>1, d1=-0.001, d2=-0.001 g=-0.001
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
>2, d1=-0.001, d2=0.001 g=0.001
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
>3, d1=0.001, d2=0.003 g=0.003
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
>4, d1=0.002, d2=0.004 g=0.004
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
>5, d1=0.004, d2=0.005 g=0.005
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
>6, d1=0.005, d2=0.006 g=0.006
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
>7, d1=0.006, d2=0.008 g=0.008
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
>8, d1=0.007, d2=0.009 g=0.009
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
>9, d1=0.008, d2=0.010 g=0.010
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
>10, d1=0.009, d2=0.011 g=0.011
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
>11, d1=0.010, d2=0.011 g=0.011
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
>12, d1=0.011, d2=0.012 g=0.012
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
>13, d1=0.011, d2=0.013 g=0.013
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
>14, d1=0.012, d2=0.013 g=0.013
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
>15, d1=0.013, d2=0.014 g=0.014
1/1 

>Saved: plot_004x004-tuned.png and model_004x004-tuned.h5
Scaled Data (1000, 8, 8, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
>1, d1=-0.001, d2=0.606 g=0.606
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
>2, d1=0.404, d2=0.606 g=0.606
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
>3, d1=0.485, d2=0.606 g=0.606
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
>4, d1=0.520, d2=0.606 g=0.606
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
>5, d1=0.539, d2=0.606 g=0.606
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
>6, d1=0.551, d2=0.605 g=0.605
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
>7, d1=0.559, d2=0.605 g=0.605
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
>8, d1=0.565, d2=0.605 g=0.605
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
>9, d1=0.569, d2=0.604 g=0.604
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
>10, d1=0.572, d2=0.604 g=0.604
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
>11, d1=0.575, d2=0.603 g=0.603
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
>12, d1=0.577, d2=0.603 g=0.603
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
>13, d1=0.579, d2=0.603 g=0.603
1/1 ━━━━━━━━━━━

>Saved: plot_008x008-faded.png and model_008x008-faded.h5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
>1, d1=-0.001, d2=0.097 g=0.097
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
>2, d1=0.065, d2=0.099 g=0.099
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
>3, d1=0.079, d2=0.099 g=0.099
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
>4, d1=0.085, d2=0.100 g=0.100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
>5, d1=0.089, d2=0.100 g=0.100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
>6, d1=0.091, d2=0.100 g=0.100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
>7, d1=0.093, d2=0.101 g=0.101
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
>8, d1=0.094, d2=0.101 g=0.101
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
>9, d1=0.095, d2=0.101 g=0.101
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
>10, d1=0.096, d2=0.101 g=0.101
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
>11, d1=0.096, d2=0.102 g=0.102
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
>12, d1=0.097, d2=0.102 g=0.102
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
>13, d1=0.098, d2=0.102 g=0.102
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
>14,

>Saved: plot_008x008-tuned.png and model_008x008-tuned.h5
Scaled Data (1000, 16, 16, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
>1, d1=-0.001, d2=0.201 g=0.201
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
>2, d1=0.134, d2=0.201 g=0.201
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
>3, d1=0.161, d2=0.201 g=0.201
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
>4, d1=0.172, d2=0.200 g=0.200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
>5, d1=0.178, d2=0.200 g=0.200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
>6, d1=0.182, d2=0.199 g=0.199
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
>7, d1=0.184, d2=0.199 g=0.199
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
>8, d1=0.186, d2=0.199 g=0.199
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
>9, d1=0.187, d2=0.198 g=0.198
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
>10, d1=0.188, d2=0.198 g=0.198
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
>11, d1=0.188, d2=0.197 g=0.197
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
>12, d1=0.189, d2=0.197 g=0.197
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
>13, d1=0.189, d2=0.197 g=0.197
1/1 ━━━━━

>Saved: plot_016x016-faded.png and model_016x016-faded.h5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
>1, d1=-0.000, d2=0.031 g=0.031
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
>2, d1=0.021, d2=0.031 g=0.031
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
>3, d1=0.025, d2=0.031 g=0.031
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
>4, d1=0.027, d2=0.031 g=0.031
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
>5, d1=0.028, d2=0.031 g=0.031
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
>6, d1=0.029, d2=0.031 g=0.031
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
>7, d1=0.029, d2=0.032 g=0.032
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
>8, d1=0.029, d2=0.032 g=0.032
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
>9, d1=0.030, d2=0.032 g=0.032
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
>10, d1=0.030, d2=0.032 g=0.032
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
>11, d1=0.030, d2=0.032 g=0.032
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
>12, d1=0.030, d2=0.032 g=0.032
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
>13, d1=0.030, d2=0.032 g=0.032
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
>

>Saved: plot_016x016-tuned.png and model_016x016-tuned.h5
Scaled Data (1000, 32, 32, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
>1, d1=-0.000, d2=0.059 g=0.059
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
>2, d1=0.039, d2=0.059 g=0.059
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
>3, d1=0.047, d2=0.059 g=0.059
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
>4, d1=0.050, d2=0.059 g=0.059
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
>5, d1=0.052, d2=0.059 g=0.059
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
>6, d1=0.053, d2=0.059 g=0.059
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
>7, d1=0.054, d2=0.058 g=0.058
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
>8, d1=0.055, d2=0.058 g=0.058
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
>9, d1=0.055, d2=0.058 g=0.058
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
>10, d1=0.055, d2=0.058 g=0.058
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
>11, d1=0.056, d2=0.058 g=0.058
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
>12, d1=0.056, d2=0.058 g=0.058
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
>13, d1=0.056, d2=0.058 g=0.058
1/1 ━━━━━